In [73]:
# Импорт необходимых библиотек
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE

In [63]:
# Загрузка данных для тренировки
train_data = pd.read_csv('/content/train.csv')

In [64]:
# Очистка от NaN
train_data = train_data.dropna()

# Применение простой модели

In [26]:
# Разделение данных на тренировочный и валидационный наборы
X_train, X_val, y_train, y_val = train_test_split(train_data['text'], train_data['label'], test_size=0.2, random_state=42)

# Создание и обучение модели
model = make_pipeline(CountVectorizer(), MultinomialNB())
model.fit(X_train, y_train)

# Предсказание на валидационном наборе
val_predictions = model.predict(X_val)

# Оценка качества модели с использованием метрики F1
f1 = f1_score(y_val, val_predictions)

# Вывод результата метрики F1
print(f"F1 Score on Validation Set: {f1}")

F1 Score on Validation Set: 0.6960095906833362


# Применение XGBoost


In [31]:
# Разделение данных на тренировочный и валидационный наборы
X_train, X_val, y_train, y_val = train_test_split(train_data['text'], train_data['label'], test_size=0.2, random_state=42)

# Создание и обучение модели с использованием TfidfVectorizer и XGBoost
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)

model = XGBClassifier(random_state=42)
model.fit(X_train_tfidf, y_train)

# Предсказание на валидационном наборе
val_predictions = model.predict(X_val_tfidf)

# Оценка качества модели на валидационном наборе
f1 = f1_score(y_val, val_predictions)
print(f"F1 Score on Validation Set: {f1}")

F1 Score on Validation Set: 0.6232937422442829


In [ ]:
# Создание пайплайна с TfidfVectorizer и XGBoost
pipeline = make_pipeline(TfidfVectorizer(), XGBClassifier(random_state=42))

# Параметры для подбора
param_grid = {
    'xgbclassifier__n_estimators': [50, 100, 200],
    'xgbclassifier__max_depth': [3, 5, 7],
    'xgbclassifier__learning_rate': [0.01, 0.1, 0.2],
}

# Подбор параметров с использованием Grid Search
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1', verbose=1)
grid_search.fit(X_train, y_train)

# Вывод лучших параметров
print("Best Parameters:", grid_search.best_params_)

In [ ]:
# Аугментация данных для балансировки классов
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train.values.reshape(-1, 1), y_train)

# Обучение модели на всем тренировочном наборе с лучшими параметрами
grid_search.best_estimator_.fit(X_train_resampled.flatten(), y_train_resampled)

# Предсказание на валидационном наборе
val_predictions = grid_search.best_estimator_.predict(X_val)

# Вывод матрицы ошибок и отчета о классификации
print("Confusion Matrix:\n", confusion_matrix(y_val, val_predictions))
print("\nClassification Report:\n", classification_report(y_val, val_predictions))

# Оценка качества модели на валидационном наборе
f1 = f1_score(y_val, val_predictions)
print(f"\nF1 Score on Validation Set: {f1}")

# Применение кросс-валидации

In [65]:
# Создание и обучение модели с использованием кросс-валидации
model = make_pipeline(TfidfVectorizer(), LogisticRegression())
cross_val_scores = cross_val_score(model, train_data['text'], train_data['label'], cv=5, scoring='f1')

# Вывод средней оценки F1 на кросс-валидации
print(f"Mean F1 Score on Cross-Validation: {cross_val_scores.mean()}")

Mean F1 Score on Cross-Validation: 0.7140721841279575


In [66]:
# Загрузка данных для тестирования
test_data = pd.read_csv('/content/test.csv', delimiter=',')

In [67]:
test_data = test_data.dropna()

In [68]:
# Обучение TF-IDF векторайзера на тренировочных данных
model.fit(train_data['text'], train_data['label'])

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('logisticregression', LogisticRegression())])

In [69]:
# Предсказание на тестовом наборе
test_predictions = model.predict(test_data['text'])

In [77]:
# Создание DataFrame с предсказаниями
prediction_df = pd.DataFrame({'index': test_data['index'], 'label': test_predictions})

In [78]:
# Сохранение предсказаний в файл prediction.csv
prediction_df.to_csv('prediction.csv', index=False, sep=';')

In [80]:
df = pd.read_csv('/content/prediction.csv')
df.head()

,index;label
0,61947;0.0
1,208586;0.0
2,187819;0.0
3,5277;0.0
4,149042;1.0
